# Test

In [1]:
import molsysmt as msm

In [2]:
molsys = msm.systems.demo['TcTIM']['1tcd.mmtf']

In [3]:
molsys = msm.convert(molsys)

In [4]:
msm.get(molsys, element='atom', chain_id=True)

array(['A', 'A', 'A', ..., 'D', 'D', 'D'], dtype=object)

In [ ]:
msm.set(molsys, element='atom', chain_id='X')

In [ ]:
msm.get(molsys, element='atom', chain_id=True)

In [ ]:
msm.info(molsys, element='chain')

In [5]:
molsys.topology.atoms_dataframe['chain_id'].unique()

array(['A', 'B', 'C', 'D'], dtype=object)

In [11]:
molsys.topology.atoms_dataframe.loc[100, 'chain_id']='X'

In [13]:
aux = molsys.topology.atoms_dataframe['chain_id'].unique()

In [15]:
for ii,jj in enumerate(aux):
    print(ii,jj)

0 A
1 X
2 B
3 C
4 D
